# ASL Word — Live Webcam Testing
# اختبار كلمات لغة الإشارة الإنجليزية — بث مباشر من الكاميرا

This notebook lets you **test your trained ASL word model in real-time** using your webcam.

هذا الدفتر يتيح لك **اختبار نموذج كلمات لغة الإشارة مباشرةً** باستخدام كاميرا الويب.

### How it works / كيف يعمل:
1. **Continuous capture** → MediaPipe extracts hand landmarks every frame / التقاط مستمر → MediaPipe يستخرج نقاط اليد
2. **Sliding window** → buffers the last 30 frames into a sequence / نافذة منزلقة → يخزن آخر 30 إطار في تسلسل
3. **Prediction** → feeds the sequence to the BiLSTM model every 0.5s / التنبؤ → يغذي التسلسل للنموذج كل 0.5 ثانية
4. **Sentence building** → confirmed words are appended to a sentence / بناء الجملة → الكلمات المؤكدة تُضاف للجملة

### Controls / التحكم:
| Key / مفتاح | Action / الإجراء |
|---|---|
| `q` | Quit / خروج |
| `r` | Reset sentence / مسح الجملة |
| `SPACE` | Add space between words / إضافة مسافة |
| `BACKSPACE` | Delete last word / حذف آخر كلمة |

### Requirements / المتطلبات:
- Trained model: `asl_word_lstm_model_best.h5`
- Class mapping: `asl_word_classes.csv`
- Webcam connected / كاميرا متصلة

In [ ]:
# ===============================
# CELL 1: IMPORTS & SETUP
# الخلية 1: الاستيرادات والإعدادات
# ===============================

import cv2
import json
import time
import numpy as np
import pandas as pd
import mediapipe as mp
import tensorflow as tf
from pathlib import Path
from collections import deque

print(f'TensorFlow: {tf.__version__}')
print(f'OpenCV: {cv2.__version__}')
print(f'MediaPipe: {mp.__version__}')

# Check GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f'✅ GPU detected: {gpus[0].name}')
else:
    print('⚠️ No GPU — running on CPU')


In [ ]:
# ===============================
# CELL 2: CONFIGURATION
# الخلية 2: الإعدادات
# ===============================

PROJECT_ROOT = Path(r'E:/Term 9/Grad')
SLR_MAIN = PROJECT_ROOT / 'Main/Sign-Language-Recognition-System-main/SLR Main'
WORDS_ROOT = SLR_MAIN / 'Words'
OUTPUT_DIR = WORDS_ROOT / 'ASL Word (English)'
SHARED_CSV = WORDS_ROOT / 'Shared/shared_word_vocabulary.csv'

# Model files
MODEL_PATH = OUTPUT_DIR / 'asl_word_lstm_model_best.h5'
CLASSES_CSV = OUTPUT_DIR / 'asl_word_classes.csv'

# Sequence parameters (must match training)
SEQUENCE_LENGTH = 30    # frames per sequence
NUM_FEATURES = 63       # 21 landmarks × 3 (x, y, z)

# Live inference settings
CONFIDENCE_THRESHOLD = 0.35     # minimum confidence to accept a prediction
PREDICTION_INTERVAL = 0.5       # seconds between predictions
STABILITY_WINDOW = 3            # consecutive same predictions needed to confirm
COOLDOWN_TIME = 2.0             # seconds after confirming a word before next

# Camera
CAMERA_INDEX = 0
CAMERA_WIDTH = 1280
CAMERA_HEIGHT = 720

print(f'📂 Model  : {MODEL_PATH}')
print(f'📂 Classes: {CLASSES_CSV}')
print(f'🎬 Sequence: {SEQUENCE_LENGTH} frames × {NUM_FEATURES} features')
print(f'🎯 Confidence threshold: {CONFIDENCE_THRESHOLD}')
print(f'🔁 Stability window: {STABILITY_WINDOW} predictions')


In [ ]:
# ===============================
# CELL 3: LOAD MODEL & VOCABULARY
# الخلية 3: تحميل النموذج والمفردات
# ===============================

# --- Custom layer needed for model loading ---
class TemporalAttention(tf.keras.layers.Layer):
    """Temporal attention layer (must match training definition)."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(input_shape[1], 1),
                                 initializer='zeros', trainable=True)

    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x, self.W) + self.b)
        a = tf.nn.softmax(e, axis=1)
        output = tf.reduce_sum(x * a, axis=1)
        return output

# Load model
print('Loading model...')
model = tf.keras.models.load_model(
    str(MODEL_PATH),
    custom_objects={'TemporalAttention': TemporalAttention}
)
print(f'✅ Model loaded: {model.name} — {model.count_params():,} parameters')

# Load class mapping
class_df = pd.read_csv(CLASSES_CSV)
vocab_df = pd.read_csv(SHARED_CSV)
vocab_df = vocab_df.dropna(subset=['wlasl_class'])

id_to_english = dict(zip(vocab_df['word_id'].astype(int), vocab_df['english']))
id_to_arabic  = dict(zip(vocab_df['word_id'].astype(int), vocab_df['arabic']))
id_to_category = dict(zip(vocab_df['word_id'].astype(int), vocab_df['category']))

# Build model_index → word name mapping
index_to_word = {}
index_to_arabic = {}
for _, row in class_df.iterrows():
    idx = int(row['model_class_index'])
    wid = int(row['word_id'])
    index_to_word[idx] = id_to_english.get(wid, f'word_{wid}')
    index_to_arabic[idx] = id_to_arabic.get(wid, '')

num_classes = len(index_to_word)
print(f'🏷️ {num_classes} word classes loaded')
print(f'\n📋 Sample words: {list(index_to_word.values())[:15]}')


In [ ]:
# ===============================
# CELL 4: MEDIAPIPE HAND DETECTOR
# الخلية 4: كاشف اليد بـ MediaPipe
# ===============================

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

def extract_landmarks(frame):
    """Extract 21 hand landmarks (63 features) from a single frame.
    Returns numpy array of shape (63,) or zeros if no hand detected."""
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        lm = results.multi_hand_landmarks[0]
        vec = np.array([[p.x, p.y, p.z] for p in lm.landmark], dtype=np.float32).flatten()
        return vec, results.multi_hand_landmarks[0]
    return np.zeros(NUM_FEATURES, dtype=np.float32), None

print('✅ MediaPipe hand detector ready')


In [ ]:
# ===============================
# CELL 5: LIVE WEBCAM TESTING
# الخلية 5: الاختبار المباشر بالكاميرا
# ===============================
# Run this cell to start the live webcam feed.
# شغّل هذه الخلية لبدء البث المباشر من الكاميرا.

def run_live_test():
    """Main live testing loop with sliding window prediction."""

    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_HEIGHT)

    if not cap.isOpened():
        print('❌ Cannot open camera!')
        return

    print('📹 Camera opened. Press Q to quit, R to reset, SPACE to add space, BACKSPACE to delete.')

    # --- State variables ---
    frame_buffer = deque(maxlen=SEQUENCE_LENGTH)     # sliding window of landmark frames
    prediction_history = deque(maxlen=STABILITY_WINDOW)  # recent predictions for stability
    sentence_words = []                                # built sentence
    current_word = ''                                  # current detected word
    current_conf = 0.0                                 # current confidence
    last_prediction_time = 0.0
    last_confirmed_time = 0.0
    hand_detected = False
    fps_history = deque(maxlen=30)

    # Colors
    GREEN = (0, 200, 0)
    RED = (0, 0, 200)
    BLUE = (200, 100, 0)
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    YELLOW = (0, 220, 220)
    ORANGE = (0, 140, 255)

    while True:
        frame_start = time.time()
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]

        # --- Extract landmarks ---
        landmarks, hand_lm = extract_landmarks(frame)
        hand_detected = hand_lm is not None
        frame_buffer.append(landmarks)

        # --- Draw hand landmarks ---
        if hand_lm is not None:
            mp_drawing.draw_landmarks(
                frame, hand_lm, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

        # --- Predict when buffer is full ---
        now = time.time()
        if len(frame_buffer) == SEQUENCE_LENGTH and (now - last_prediction_time) >= PREDICTION_INTERVAL:
            last_prediction_time = now

            # Build sequence
            seq = np.array(list(frame_buffer), dtype=np.float32)
            seq = np.expand_dims(seq, axis=0)  # (1, 30, 63)

            # Check if sequence has enough non-zero frames
            non_zero = np.sum(np.any(seq[0] != 0, axis=1))
            if non_zero >= SEQUENCE_LENGTH * 0.3:  # at least 30% non-zero frames
                proba = model.predict(seq, verbose=0)[0]
                pred_idx = np.argmax(proba)
                pred_conf = proba[pred_idx]
                pred_word = index_to_word.get(pred_idx, '?')

                # Top-3 for display
                top3_idx = np.argsort(proba)[-3:][::-1]
                top3 = [(index_to_word.get(i, '?'), proba[i]) for i in top3_idx]

                if pred_conf >= CONFIDENCE_THRESHOLD:
                    current_word = pred_word
                    current_conf = pred_conf
                    prediction_history.append(pred_word)

                    # Check stability: same word predicted N times in a row
                    if (len(prediction_history) == STABILITY_WINDOW and
                        len(set(prediction_history)) == 1 and
                        (now - last_confirmed_time) >= COOLDOWN_TIME):
                        # Confirm the word!
                        sentence_words.append(current_word)
                        last_confirmed_time = now
                        prediction_history.clear()
                        print(f'✅ Confirmed: "{current_word}" ({current_conf:.1%})')
                else:
                    current_word = ''
                    current_conf = 0.0
            else:
                current_word = ''
                current_conf = 0.0

        # --- Draw UI Overlay ---

        # Top bar: prediction info
        cv2.rectangle(frame, (0, 0), (w, 90), BLACK, -1)
        cv2.rectangle(frame, (0, 0), (w, 90), WHITE, 2)

        if current_word:
            color = GREEN if current_conf >= 0.6 else YELLOW if current_conf >= 0.4 else ORANGE
            cv2.putText(frame, f'Word: {current_word}', (15, 35),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
            cv2.putText(frame, f'Confidence: {current_conf:.1%}', (15, 65),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Confidence bar
            bar_x = 450
            bar_w = 200
            bar_h = 20
            cv2.rectangle(frame, (bar_x, 20), (bar_x + bar_w, 20 + bar_h), (50, 50, 50), -1)
            fill_w = int(bar_w * current_conf)
            cv2.rectangle(frame, (bar_x, 20), (bar_x + fill_w, 20 + bar_h), color, -1)
            cv2.rectangle(frame, (bar_x, 20), (bar_x + bar_w, 20 + bar_h), WHITE, 1)

            # Stability progress
            stable_count = sum(1 for p in prediction_history if p == current_word)
            cv2.putText(frame, f'Stability: {stable_count}/{STABILITY_WINDOW}',
                        (bar_x, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.6, WHITE, 1)
        else:
            status = 'Show a sign...' if hand_detected else 'No hand detected'
            cv2.putText(frame, status, (15, 45),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (150, 150, 150), 2)

        # Top-3 predictions (right side)
        if current_word and 'top3' in dir():
            tx = w - 320
            cv2.putText(frame, 'Top 3:', (tx, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 1)
            for rank, (tw, tc) in enumerate(top3):
                y_pos = 45 + rank * 20
                cv2.putText(frame, f'{rank+1}. {tw} ({tc:.1%})', (tx, y_pos),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 1)

        # Bottom bar: sentence
        sentence_text = ' '.join(sentence_words) if sentence_words else '(sentence will appear here)'
        cv2.rectangle(frame, (0, h - 55), (w, h), BLACK, -1)
        cv2.rectangle(frame, (0, h - 55), (w, h), WHITE, 2)
        cv2.putText(frame, f'Sentence: {sentence_text}', (15, h - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, WHITE, 2)

        # Buffer indicator (bottom-left)
        buf_fill = len(frame_buffer) / SEQUENCE_LENGTH
        buf_color = GREEN if buf_fill >= 1.0 else YELLOW
        cv2.putText(frame, f'Buffer: {len(frame_buffer)}/{SEQUENCE_LENGTH}',
                    (15, h - 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, buf_color, 1)

        # Hand status indicator
        hand_color = GREEN if hand_detected else RED
        hand_text = 'HAND OK' if hand_detected else 'NO HAND'
        cv2.circle(frame, (w - 80, h - 75), 8, hand_color, -1)
        cv2.putText(frame, hand_text, (w - 170, h - 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, hand_color, 1)

        # FPS counter
        fps = 1.0 / max(time.time() - frame_start, 1e-6)
        fps_history.append(fps)
        avg_fps = sum(fps_history) / len(fps_history)
        cv2.putText(frame, f'FPS: {avg_fps:.0f}', (w - 110, 115),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, WHITE, 1)

        # Cooldown indicator
        cooldown_remaining = max(0, COOLDOWN_TIME - (now - last_confirmed_time))
        if cooldown_remaining > 0:
            cv2.putText(frame, f'Cooldown: {cooldown_remaining:.1f}s',
                        (w // 2 - 80, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.6, ORANGE, 2)

        # --- Show frame ---
        cv2.imshow('ASL Word Recognition — Live Test', frame)

        # --- Handle keyboard ---
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):
            sentence_words.clear()
            prediction_history.clear()
            current_word = ''
            print('🔄 Sentence reset')
        elif key == 32:  # SPACE
            sentence_words.append(' ')
            print('   [space added]')
        elif key == 8:   # BACKSPACE
            if sentence_words:
                removed = sentence_words.pop()
                print(f'⬅️ Removed: "{removed}"')

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

    final_sentence = ' '.join(sentence_words)
    print(f'\n📝 Final sentence: {final_sentence}')
    return final_sentence

# --- RUN ---
result = run_live_test()


## Tips / نصائح

| Issue / المشكلة | Solution / الحل |
|---|---|
| **Low FPS** | Close other apps, reduce `CAMERA_WIDTH`/`CAMERA_HEIGHT` / قلل دقة الكاميرا |
| **Wrong predictions** | Hold the sign steadily for ~2 seconds / ثبّت الإشارة لمدة ثانيتين |
| **Camera not opening** | Change `CAMERA_INDEX` to 1 or 2 / غيّر رقم الكاميرا |
| **Too sensitive** | Increase `STABILITY_WINDOW` to 4-5 / زوّد نافذة الاستقرار |
| **Not detecting** | Lower `CONFIDENCE_THRESHOLD` to 0.25 / قلل حد الثقة |
| **Too slow between words** | Decrease `COOLDOWN_TIME` to 1.0 / قلل وقت الانتظار |

### How to perform a sign / كيف تؤدي إشارة:
1. Face the camera with your hand clearly visible / وجّه يدك للكاميرا بوضوح
2. Perform the sign gesture smoothly / أدِّ حركة الإشارة بسلاسة
3. Wait for the stability bar to fill up / انتظر حتى يمتلئ شريط الاستقرار
4. The word will be confirmed and added to the sentence / الكلمة ستُؤكد وتُضاف للجملة